In [5]:
from numpy import sort
from src.Models.SpectralTransformer import SpectralTransformer
from src.model.model import MyModel as NewModel
from src.model.model import MyBigModel as NewBigModel
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr
# from skimage.metrics import structural_similarity as ssim
import torch
from src.DataManipulation.DataLoader import get_dataloaders
import os

model_path = "../best.pth"
raw_dir = "../uw_data/uw_data/train/a"
ref_dir = "../uw_data/uw_data/train/b"
test_raw_dir = "../uw_data/uw_data/test/a"
test_ref_dir = "../uw_data/uw_data/test/b"
batch_size=1
num_workers=4
_, test_loader = get_dataloaders(raw_dir,ref_dir,test_raw_dir,test_ref_dir,batch_size=batch_size,num_workers=num_workers)

device = "cuda"
model = SpectralTransformer().to(device)
#model = NewModel().to(device)
#model = NewBigModel().to(device)
#checkpoint = torch.load(model_path, map_location=device)
#model.load_state_dict(checkpoint['model_state_dict'])
model = torch.load(model_path, map_location=device, weights_only=False)
model.eval()

psnr_values = []
ssim_values = 0

results_dir = 'evaluation_results'
os.makedirs(results_dir, exist_ok=True)

with torch.no_grad():
    for i, (raw_img, ref_img) in enumerate(test_loader):
        raw_img = raw_img.to(device)
        ref_img = ref_img.to(device)

        enhanced_img = model(raw_img)

        enhanced_np = enhanced_img[0].squeeze().detach().cpu().numpy()
        enhanced_np = enhanced_np.transpose((1, 2, 0))
        ref_np = ref_img.squeeze().detach().cpu().numpy()
        ref_np = np.transpose(ref_np, (1, 2, 0))
        raw_np = raw_img
        print("enhanced_np.shape:", enhanced_np.shape)
        print("ref_np.shape:", ref_np.shape)
        curr_psnr = psnr(ref_np, enhanced_np)
        # curr_ssim = torchSSIM(ref_np, enhanced_np)
        # curr_psnr = curr_psnr.cpu()
        psnr_values.append(curr_psnr)
        # ssim_values += curr_ssim

        # Print progress
        if (i + 1) % 20 == 0:
            print(f"Processed {i + 1}/{len(test_loader)} test images")

psnr_values = np.array(psnr_values)
psnr_values = sort(psnr_values)
psnr_values = psnr_values[:]
print(len(psnr_values))
avg_psnr = psnr_values.sum()/len(psnr_values)
# avg_ssim = ssim_values/len(test_loader)

print(f"Evaluation Results:")
print(f"Average PSNR: {avg_psnr:.2f} dB")
# print(f"Average SSIM: {avg_ssim:.4f}")

with open(f'{results_dir}/metrics.txt', 'a') as f:
    f.write(f"Model: {model_path}\n")
    f.write(f"Average PSNR: {avg_psnr:.2f} dB\n")
    # f.write(f"Average SSIM: {avg_ssim:.4f}\n")

ModuleNotFoundError: No module named 'model_without_CA'

In [2]:
import torch
import torch.nn.functional as F
import numpy as np
import math
from PIL import Image
import cv2
from pytorch_msssim import ssim

def torchPSNR(tar_img, prd_img):
    imdff = torch.clamp(prd_img, 0, 1) - torch.clamp(tar_img, 0, 1)
    rmse = (imdff**2).mean().sqrt()
    ps = 20*torch.log10(1/rmse)
    return ps

def torchSSIM(tar_img, prd_img):
    return ssim(tar_img, prd_img, data_range=1.0, size_average=True)



In [ ]:
from src.Models.SpectralTransformer import SpectralTransformer

model = SpectralTransformer()

